In [29]:
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.layers import LSTM, GRU, Bidirectional, LayerNormalization, Conv1D, Dense, Concatenate, Dropout, TimeDistributed
from tensorflow.keras.optimizers import Adam

class PhysicsInformedNN(tf.keras.Model):
    def __init__(self,
                 seq_length=30,
                 num_features=23,
                 use_black_scholes=False,
                 risk_free_rate=0.0001,
                 strike_idx=3,
                 maturity_idx=4,
                 dropout_rate=0.2):
        super().__init__()
        assert seq_length >= 2, "Sequence length must be at least 2"
        self.seq_length = seq_length
        self.num_features = num_features
        self.use_black_scholes = use_black_scholes
        self.risk_free_rate = risk_free_rate
        self.strike_idx = strike_idx
        self.maturity_idx = maturity_idx
        # Adaptive physics weight network
        self.weight_net = tf.keras.Sequential([
            Dense(16, activation='relu', input_shape=(3,)),
            Dense(1, activation='sigmoid')
        ])
        # Curriculum phase flag
        self.curr_phase = tf.Variable(0, trainable=False, dtype=tf.int32)
        # Layers
        self.lstm_branch = Bidirectional(LSTM(64, return_sequences=True))
        self.norm_branch = LayerNormalization()
        self.conv_branch = tf.keras.Sequential([
            Conv1D(32, 3, activation='relu', padding='same'),
            Bidirectional(GRU(32, return_sequences=True)),
            Conv1D(64, 1, padding='same')
        ])
        self.fusion = Dense(64, activation='relu')
        self.dropout = Dropout(dropout_rate)
        # Output heads
        self.mean_head = TimeDistributed(Dense(1), name='mean_head')
        self.logvar_head = TimeDistributed(Dense(1), name='logvar_head')
        self.vol_head = TimeDistributed(Dense(1), name='vol_head')
        # Moving averages
        self.loss_ma = {'nll': tf.Variable(1.0, trainable=False),
                        'phys': tf.Variable(1.0, trainable=False),
                        'bs': tf.Variable(1.0, trainable=False)}
        # Metrics
        self.nll_tracker = tf.keras.metrics.Mean(name='nll')
        self.phys_tracker = tf.keras.metrics.Mean(name='phys_loss')
        self.total_tracker = tf.keras.metrics.Mean(name='loss')

    @property
    def metrics(self):
        return [self.nll_tracker, self.phys_tracker, self.total_tracker]

    def call(self, inputs, training=False):
        h1 = self.lstm_branch(inputs, training=training)
        h1 = self.norm_branch(h1, training=training)
        h2 = self.conv_branch(inputs, training=training)
        h = Concatenate()([h1, h2])
        h = self.fusion(h)
        h = self.dropout(h, training=training)
        mu_seq = self.mean_head(h)
        logvar_seq = self.logvar_head(h)
        vol_seq = self.vol_head(h)
        return mu_seq, logvar_seq, vol_seq

    def train_step(self, data):
        x, y_true = data
        y_true = tf.reshape(y_true, (-1,1))
        with tf.GradientTape() as tape:
            mu_seq, logvar_seq, vol_seq = self(x, training=True)
            mu = mu_seq[:, -1, :]
            logvar = tf.clip_by_value(logvar_seq[:, -1, :], -10.0, 10.0)
            var = tf.clip_by_value(tf.exp(logvar), 1e-6, 1e6)
            # Loss components
            mse_loss = tf.reduce_mean(tf.square(y_true - mu))
            nll_loss = 0.5 * tf.reduce_mean(logvar + tf.square(y_true - mu) / var)
            phys_loss = tf.math.nan_to_num(self._physics_loss(mu_seq, x))
            bs_loss = tf.nan_to_num(self._bs_penalty(mu, x)) if self.use_black_scholes else 0.0
            # Adaptive weight
            norms_vec = tf.stack([nll_loss / self.loss_ma['nll'], phys_loss / self.loss_ma['phys'], bs_loss / self.loss_ma['bs']], axis=0)
            norms = tf.reshape(norms_vec, (1, 3))
            physics_w = self.weight_net(norms)[0, 0]
            # Auxiliary volatility supervision
            vol_true = tf.math.sqrt(tf.math.reduce_variance(x[:,:,4], axis=1, keepdims=True))
            aux_loss = tf.reduce_mean(tf.square(vol_seq[:,-1,:] - vol_true))
            # Total loss by phase
            phase = self.curr_phase.read_value()
            def phase0(): return mse_loss
            def phase1(): return nll_loss + phys_loss + aux_loss
            def phase2(): return nll_loss + physics_w * phys_loss + bs_loss + aux_loss
            total_loss = tf.switch_case(phase, branch_fns={0: phase0, 1: phase1, 2: phase2}, default=phase2)
        grads = tape.gradient(total_loss, self.trainable_variables)
        grads = [tf.clip_by_norm(g, 1.0) for g in grads]
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        # Update moving averages
        self.loss_ma['nll'].assign(0.99 * self.loss_ma['nll'] + 0.01 * nll_loss)
        self.loss_ma['phys'].assign(0.99 * self.loss_ma['phys'] + 0.01 * phys_loss)
        self.loss_ma['bs'].assign(0.99 * self.loss_ma['bs'] + 0.01 * bs_loss)
        # Track metrics
        self.nll_tracker.update_state(nll_loss)
        self.phys_tracker.update_state(phys_loss)
        self.total_tracker.update_state(total_loss)
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        x, y_true = data
        y_true = tf.reshape(y_true, (-1,1))
        mu_seq, logvar_seq, _ = self(x, training=False)
        mu = mu_seq[:, -1, :]
        logvar = tf.clip_by_value(logvar_seq[:, -1, :], -10.0, 10.0)
        var = tf.clip_by_value(tf.exp(logvar), 1e-6, 1e6)
        nll_loss = 0.5 * tf.reduce_mean(logvar + tf.square(y_true - mu) / var)
        phys_loss = tf.nan_to_num(self._physics_loss(mu_seq, x))
        bs_loss = tf.nan_to_num(self._bs_penalty(mu, x)) if self.use_black_scholes else 0.0
        total_loss = nll_loss + phys_loss + bs_loss
        self.nll_tracker.update_state(nll_loss)
        self.phys_tracker.update_state(phys_loss)
        self.total_tracker.update_state(total_loss)
        return {m.name: m.result() for m in self.metrics}

    def _physics_loss(self, mu_seq, inputs):
        returns = mu_seq[:,1:,:] - mu_seq[:,:-1,:]
        comp_vol = tf.math.reduce_std(returns, axis=1)
        upper = inputs[:,-1,19]
        lower = inputs[:,-1,20]
        v = tf.squeeze(comp_vol)
        vol_l = tf.reduce_mean(tf.square(tf.nn.relu(upper - v) + tf.nn.relu(v - lower)))
        r = tf.squeeze(returns, -1)
        vchg = inputs[:,1:,4] - inputs[:,:-1,4]
        r_m = tf.reduce_mean(r,1,keepdims=True); v_m = tf.reduce_mean(vchg,1,keepdims=True)
        cov = tf.reduce_mean((r - r_m) * (vchg - v_m), axis=1)
        corr = cov / (tf.math.reduce_std(r,1) * tf.math.reduce_std(vchg,1) + 1e-7)
        pv_l = tf.reduce_mean(tf.square(tf.nn.relu(0.5 - corr)))
        P = mu_seq[:, :-1, :]
        drift_res = returns - self.risk_free_rate * P
        drift_l = tf.reduce_mean(tf.square(tf.reduce_mean(drift_res, axis=1)))
        data_vol = (upper + lower) * 0.5
        diff_res = tf.math.reduce_variance(returns, axis=1) - data_vol**2
        diff_l = tf.reduce_mean(diff_res**2)
        return vol_l * 0.55 + pv_l * 0.43 + drift_l * 0.02 + diff_l

    def _bs_penalty(self, mu, inputs):
        S = tf.squeeze(mu, -1)
        sigma = (inputs[:,-1,19] + inputs[:,-1,20]) * 0.5
        K = inputs[:,-1,self.strike_idx]
        T = inputs[:,-1,self.maturity_idx]
        r = self.risk_free_rate
        sqrtT = tf.sqrt(T) + 1e-7
        d1 = (tf.math.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * sqrtT)
        d2 = d1 - sigma * sqrtT
        normal = tfp.distributions.Normal(0.,1.)
        BS = S * normal.cdf(d1) - K * tf.exp(-r*T) * normal.cdf(d2)
        return tf.reduce_mean(tf.square(S - BS))

    def next_phase(self):
        self.curr_phase.assign_add(1)

    def online_update(self, x, y_true, lr=1e-4):
        optimizer = tf.keras.optimizers.Adam(lr)
        with tf.GradientTape() as tape:
            mu_seq, logvar_seq, _ = self(x, training=True)
            mu = mu_seq[:, -1, :]
            logvar = tf.clip_by_value(logvar_seq[:, -1, :], -10.0, 10.0)
            var = tf.clip_by_value(tf.exp(logvar), 1e-6, 1e6)
            y_true = tf.reshape(y_true, (-1,1))
            nll_loss = 0.5 * tf.reduce_mean(logvar + tf.square(y_true - mu) / var)
        grads = tape.gradient(nll_loss, self.trainable_variables)
        optimizer.apply_gradients(zip(grads, self.trainable_variables))

    def predict_intervals(self, x, num_samples=100, q=(0.05,0.95)):
        preds = [tf.squeeze(self(x, training=True)[0], -1) for _ in range(num_samples)]
        stack = tf.stack(preds)
        return (tfp.stats.percentile(stack, q[0]*100, axis=0),
                tfp.stats.percentile(stack, q[1]*100, axis=0))

In [9]:
! pip install scikit-learn

  Using cached scikit_learn-1.6.1-cp310-cp310-win_amd64.whl.metadata (15 kB)
  Using cached scipy-1.15.2-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.6.1-cp310-cp310-win_amd64.whl (11.1 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.15.2-cp310-cp310-win_amd64.whl (41.2 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)


In [10]:
import pickle
import numpy as np
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler

class TimeSeriesPipeline:
    def __init__(self, seq_length=30, val_size=0.2, test_size=0.1):
        self.seq_length = seq_length
        self.val_size = val_size
        self.test_size = test_size
        self.scalers = {}
        self.price_cols = ['open', 'high', 'low', 'close', 'av_pr', 'diff', '7_hour_SMA', 
                           '30_hour_SMA', '7_hour_EMA', '30_hour_EMA', '12_hour_EMA', '26_hour_EMA', '20_hour_SMA']
        self.volatility_cols = ['20_hour_STD', 'Upper_Band', 'Lower_Band']
        self.momentum_cols = ['RSI', 'MACD', 'Signal_Line']
        self.lag_cols = ['lag_1', 'lag_2', 'lag_3']

    def preprocess_data(self, df, fit_scalers=True, save_scalers=True, scalers_path="scalers.pkl"):
        """
        Preprocess DataFrame and split into train (70%), validation (20%), test (10%).
        Returns scaled and unscaled splits for X and y.
        """
        print("Starting data preprocessing...")
        unscaled_data = df.copy()
        # Fit or load scalers
        if not fit_scalers:
            with open(scalers_path, 'rb') as f:
                self.scalers = pickle.load(f)
        else:
            self._fit_scalers(df, self.price_cols, self.volatility_cols,
                               self.momentum_cols, self.lag_cols)
            if save_scalers:
                with open(scalers_path, 'wb') as f:
                    pickle.dump(self.scalers, f)
        # Transform features
        df_scaled = self._transform_features(df.copy(), self.price_cols,
                                            self.volatility_cols,
                                            self.momentum_cols,
                                            self.lag_cols)
        # Create sequences
        X_scaled, y_scaled = self.create_sequences(df_scaled)
        X_unscaled, y_unscaled = self.create_sequences(unscaled_data)
        # Compute split indices
        total = len(X_scaled)
        train_end = int((1 - self.val_size - self.test_size) * total)
        val_end = int((1 - self.test_size) * total)
        # Scaled
        X_train_scaled = X_scaled[:train_end]
        y_train_scaled = y_scaled[:train_end]
        X_val_scaled = X_scaled[train_end:val_end]
        y_val_scaled = y_scaled[train_end:val_end]
        X_test_scaled = X_scaled[val_end:]
        y_test_scaled = y_scaled[val_end:]
        # Unscaled
        X_train_unscaled = X_unscaled[:train_end]
        y_train_unscaled = y_unscaled[:train_end]
        X_val_unscaled = X_unscaled[train_end:val_end]
        y_val_unscaled = y_unscaled[train_end:val_end]
        X_test_unscaled = X_unscaled[val_end:]
        y_test_unscaled = y_unscaled[val_end:]
        print(f"Data preprocessing finished. Training: {len(X_train_scaled)}, Validation: {len(X_val_scaled)}, Test: {len(X_test_scaled)}")
        return (X_train_scaled, X_val_scaled, X_test_scaled,
                y_train_scaled, y_val_scaled, y_test_scaled,
                X_train_unscaled, X_val_unscaled, X_test_unscaled,
                y_train_unscaled, y_val_unscaled, y_test_unscaled)

    def _fit_scalers(self, df, price_cols, volatility_cols, momentum_cols, lag_cols):
        self.scalers = {
            'price': RobustScaler().fit(df[price_cols]),
            'volume': RobustScaler().fit(np.log1p(df[['volume']])),
            'volatility': StandardScaler().fit(df[volatility_cols]),
            'momentum': MinMaxScaler().fit(df[momentum_cols]),
            'lag': RobustScaler().fit(df[lag_cols])
        }

    def _transform_features(self, df, price_cols, volatility_cols, momentum_cols, lag_cols):
        df[price_cols] = self.scalers['price'].transform(df[price_cols])
        df['volume'] = self.scalers['volume'].transform(np.log1p(df[['volume']]))
        df[volatility_cols] = self.scalers['volatility'].transform(df[volatility_cols])
        df[momentum_cols] = self.scalers['momentum'].transform(df[momentum_cols])
        df[lag_cols] = self.scalers['lag'].transform(df[lag_cols])
        return df

    def transform_for_prediction(self, df):
        df_transformed = self._transform_features(df.copy(),
                                                  self.price_cols,
                                                  self.volatility_cols,
                                                  self.momentum_cols,
                                                  self.lag_cols)
        if len(df_transformed) == self.seq_length:
            return np.array([df_transformed.values])
        elif len(df_transformed) > self.seq_length:
            X, _ = self.create_sequences(df_transformed)
            return X
        else:
            raise ValueError("Not enough data for a full sequence")

    def create_sequences(self, data):
        X, y = [], []
        for i in range(len(data) - self.seq_length):
            seq = data.iloc[i:i+self.seq_length].values
            target = data.iloc[i+self.seq_length]['close']
            X.append(seq)
            y.append(target)
        return np.array(X), np.array(y)

    def inverse_transform_predictions(self, pred_scaled, feature_name='close'):
        if feature_name == 'close':
            scaler = self.scalers['price']
            col_index = self.price_cols.index('close')
            pred_scaled = np.array(pred_scaled)
            return pred_scaled * scaler.scale_[col_index] + scaler.center_[col_index]
        raise ValueError(f"Unsupported feature for inversion: {feature_name}")


In [30]:
# ====================== Example Usage ======================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback

# 1. Load your raw DataFrame
df = pd.read_csv(r'C:\Users\chidi\Documents\timeseries_projects\data\xrpusdt_hourly_dataset_with_features_bn.csv')

# 2. Preprocess and split
pipeline = TimeSeriesPipeline(seq_length=30, val_size=0.2, test_size=0.1)
(X_train_s, X_val_s, X_test_s,
 y_train_s, y_val_s, y_test_s,
 X_train_u, X_val_u, X_test_u,
 y_train_u, y_val_u, y_test_u) = pipeline.preprocess_data(
    df, fit_scalers=True, save_scalers=True, scalers_path='scalers.pkl'
)

# 3. Instantiate the model
strike_idx = pipeline.price_cols.index('av_pr')
maturity_idx = pipeline.price_cols.index('diff')
model = PhysicsInformedNN(
    seq_length=30,
    num_features=len(df.columns),
    use_black_scholes=True,
    risk_free_rate=0.0001,
    strike_idx=strike_idx,
    maturity_idx=maturity_idx,
    dropout_rate=0.2
)

model.compile(optimizer=Adam(1e-3))

# 4. Define a callback to advance curriculum phases
class PhaseCallback(Callback):
    def __init__(self, switch_epochs):
        super().__init__()
        self.switch_epochs = switch_epochs

    def on_epoch_end(self, epoch, logs=None):
        if epoch + 1 in self.switch_epochs:
            print(f"Advancing to next curriculum phase at epoch {epoch+1}")
            self.model.next_phase()

phase_cb = PhaseCallback(switch_epochs=[5, 15])  # switch after 5th and 15th epochs

# 5. Train model across all phases
history = model.fit(
    X_train_s, y_train_s,
    validation_data=(X_val_s, y_val_s),
    epochs=25,
    batch_size=32,
    callbacks=[phase_cb],
    verbose=2
)

# 6. Evaluate on test set
# Point forecasts
mu_seq, logvar_seq, vol_seq = model(X_test_s, training=False)
y_pred_scaled = mu_seq[:, -1, 0]
# Inverse transform
y_pred = pipeline.inverse_transform_predictions(y_pred_scaled, feature_name='close')
y_true = y_test_u
# Compute metrics
mae = np.mean(np.abs(y_true - y_pred))
rmse = np.sqrt(np.mean((y_true - y_pred)**2))
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
print(f"MAE: {mae:.4f}, RMSE: {rmse:.4f}, MAPE: {mape:.2f}%")

# Physics and total loss metrics via evaluate
test_metrics = model.evaluate(X_test_s, y_test_s, verbose=2)
print("Test metrics (NLL, physics loss, total loss):", test_metrics)

# 7. Make predictive interval forecasts
lower_q, upper_q = model.predict_intervals(
    X_test_s, num_samples=100, q=(0.05, 0.95)
)
# Inverse transform intervals
lower = pipeline.inverse_transform_predictions(lower_q, feature_name='close')
upper = pipeline.inverse_transform_predictions(upper_q, feature_name='close')
print("First 5 lower bounds:", lower[:5])
print("First 5 upper bounds:", upper[:5])

# ====================== Visualization ======================
# 8. Overlay True vs Predicted with Intervals
plt.figure(figsize=(12,4))
plt.plot(y_true, label='True Price')
plt.plot(y_pred, label='Predicted Price')
plt.fill_between(np.arange(len(y_true)), lower, upper, color='gray', alpha=0.3, label='90% Interval')
plt.title('True vs Predicted Price with 90% Prediction Interval')
plt.xlabel('Time Index')
plt.ylabel('Price')
plt.legend()
plt.show()

# 9. Residual Histogram
residuals = y_true - y_pred
plt.figure(figsize=(6,4))
plt.hist(residuals, bins=30, edgecolor='k')
plt.title('Residuals Distribution')
plt.xlabel('Residual (True - Predicted)')
plt.ylabel('Frequency')
plt.show()

# 10. Predicted vs Actual Scatter
plt.figure(figsize=(6,6))
plt.scatter(y_true, y_pred, alpha=0.6)
minv = min(y_true.min(), y_pred.min())
maxv = max(y_true.max(), y_pred.max())
plt.plot([minv, maxv], [minv, maxv], 'r--')
plt.title('Predicted vs Actual Price')
plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')
plt.show()


Starting data preprocessing...
Data preprocessing finished. Training: 41808, Validation: 11945, Test: 5973
Epoch 1/25


c:\Users\chidi\anaconda3\envs\dl_envc\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


AttributeError: module 'tensorflow._api.v2.math' has no attribute 'nan_to_num'

In [12]:
pip install pandas numpy matplotlib

Note: you may need to restart the kernel to use updated packages.


In [5]:
! pip install tf_keras


   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 18.8 MB/s eta 0:00:00


In [3]:
! pip install tensorflow tensorflow-probability
! pip install tensorflow-addons

  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ------ --------------------------------- 1.0/7.0 MB 17.1 MB/s eta 0:00:01
   ------------ --------------------------- 2.1/7.0 MB 7.8 MB/s eta 0:00:01
   ------------ --------------------------- 2.1/7.0 MB 7.8 MB/s eta 0:00:01
   ------------------------ --------------- 4.2/7.0 MB 5.7 MB/s eta 0:00:01
   ------------------------ --------------- 4.2/7.0 MB 5.7 MB/s eta 0:00:01
   ---------------------------------------- 7.0/7.0 MB 6.0 MB/s eta 0:00:00
Using cached cloudpickle-3.1.1-py3-none-any.whl (20 kB)
   ---------------------------------------- 0.0/719.8 kB ? eta -:--:--
   --------------------------------------- 719.8/719.8 kB 14.8 MB/s eta 0:00:00
